This is the implementation of a simple FashionMNIST classifer model using PyTorch and not d2l

In [12]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [14]:
train_data = datasets.FashionMNIST(root='data', train=True, download=True, transform= ToTensor(),)
test_data = datasets.FashionMNIST(root='data',train=False, download=False, transform= ToTensor(),)

100.0%


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100.0%


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100.0%

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [29]:
class MLPScratch(nn.Module):
    def __init__(self,num_inputs, num_outputs, num_hiddens, lr, sigma=0.01):
        super().__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.num_hiddens = num_hiddens
        

        self.W1 = nn.Parameter(torch.randn(num_inputs, num_hiddens) *sigma)
        self.b1 = nn.Parameter(torch.zeros(num_hiddens))

        self.W2 = nn.Parameter(torch.randn(num_hiddens,num_outputs)*sigma)
        self.b2 = nn.Parameter(torch.zeros(num_outputs))
    def relu(self,X):
        a = torch.zeros_like(X)
        return torch.max(X,a)
        
    def forward(self, X):
        X = X.reshape((-1,self.num_inputs))
        H = self.relu(torch.matmul(X,self.W1) + self.b1)
        return torch.matmul(H,self.W2) + self.b2
        
    def loss_function(self, y_hat, y):
        log_softmax = y_hat - torch.logsumexp(y_hat, dim=1, keepdim=True)
        batch_indices = torch.arange(y_hat.size(0))
        correct_log_probs = log_softmax[batch_indices, y]
        return -correct_log_probs.mean()


In [34]:
batch_size = 256
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

device = 'cpu'
model = MLPScratch(784, 10, 256,.01).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = model(X)
        loss = model.loss_function(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        predictions = torch.argmax(logits, dim=1)
        correct += (predictions == y).sum().item()
        total += y.size(0)
print(f"Test Accuracy: {correct / total:.4f}")

Epoch 1, Loss: 1.0450
Epoch 2, Loss: 0.6036
Epoch 3, Loss: 0.5207
Epoch 4, Loss: 0.4792
Epoch 5, Loss: 0.4508
Epoch 6, Loss: 0.4344
Epoch 7, Loss: 0.4184
Epoch 8, Loss: 0.4028
Epoch 9, Loss: 0.3910
Epoch 10, Loss: 0.3812
Epoch 11, Loss: 0.3711
Epoch 12, Loss: 0.3630
Epoch 13, Loss: 0.3587
Epoch 14, Loss: 0.3511
Epoch 15, Loss: 0.3432
Test Accuracy: 0.8621


In [35]:
def predict(model, image):
    model.eval()
    with torch.no_grad():
        image = image.to(device).unsqueeze(0)  # Add batch dimension
        output = model(image)
        return torch.softmax(output, dim=1).item()
    

SyntaxError: unexpected EOF while parsing (936112332.py, line 1)